CS524: Introduction to Optimization Lecture 14
======================================

## Michael Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison

## October 4, 2024
--------------

#  Max Flow


In [1]:
import sys
import pandas as pd
import numpy as np

from gamspy import (
    Container,Set,Alias,Parameter,Variable,Equation,Model,Problem,Sense,Options,
    Domain,Number,Sum,Product,Smax,Smin,Ord,Card,SpecialValues,
    ModelStatus,SolveStatus,
)
from gamspy.exceptions import GamspyException
import math

options = Options(variable_listing_limit=100, equation_listing_limit=8)
m = Container(options=options)

# Data

In [2]:
nodes = m.addSet('nodes',records=range(1,9))
i = m.addAlias('i',nodes)
j = m.addAlias('j',nodes)
k = m.addAlias('k',nodes)
s = m.addSet('s',domain=[i],description='sources',records=[1])
t = m.addSet('t',domain=[i],description='sinks',records=[8])

u = m.addParameter('u',[i,j],description='capacities',records=[
    (1,2,8),
    (1,3,10),
    (2,4,5),
    (3,6,5),
    (4,7,4),
    (6,5,3),
    (6,8,9),
    (5,8,6),
    (7,6,2),
    (7,8,8)])
if any(u.records.min()) < 0:
    raise Exception("bad capacities given", u)

# define a dynamic set that indicates the "legal" arcs
arcs = m.addSet('arcs',domain=[i,j])
# arcs[i,j] = Number(1).where[u[i,j] > 0.0]
arcs.setRecords(u.records)

# Model

In [3]:
# VARIABLES #

x = m.addVariable("x","positive",domain=[i,j],description="flow")
    
# EQUATIONS #

balance = m.addEquation('balance',domain=[i])
balance[i].where[~s[i] & ~t[i]]= Sum(k.where[arcs[i,k]], x[i,k]) - Sum(j.where[arcs[j,i]], x[j,i]) == 0

maxflow = m.addModel(
    name="maxflow",
    equations=m.getEquations(),
    problem=Problem.LP,
    sense=Sense.MAX,
    objective=Sum(arcs[s,j], x[s,j]),
)

x.up[arcs] = u[arcs]

maxflow.solve(solver='cplex',solver_options={'lpmethod': 3, 'netfind': 1, 'preind': 0, 'names': 'no'},output=None)

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,9,7,11,LP,CPLEX,0


# Post Processing 

In [4]:
print(f"Objective Function Value: {round(maxflow.objective_value, 4)}\n")
print(f"Number of equations: {maxflow.num_equations:.0f}\n")
print("x:"); display(x.pivot())
print("balance marginals:"); display(balance.records[['i','marginal']])
print("reduced costs:"); display(x.pivot(value='marginal',fill_value=""))

Objective Function Value: 9.0

Number of equations: 7

x:


,2,3,4,5,6,7,8
1,4.0,5.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,5.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,4.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,5.0
7,0.0,0.0,0.0,0.0,0.0,0.0,4.0


balance marginals:


,i,marginal
0,2,-1.0
1,3,-1.0
2,4,-1.0
3,5,-0.0
4,6,-0.0
5,7,-0.0


reduced costs:


/Users/ferris/.venvs/gamspy++/lib/python3.12/site-packages/gams/transfer/syms/_mixins/pivot.py:272: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna(fill_value, inplace=True)


,2,3,4,5,6,7,8
1,0.0,0.0,,,,,
2,,,0.0,,,,
3,,,,,1.0,,
4,,,,,,1.0,
5,,,,,,,-0.0
6,,,,0.0,,,0.0
7,,,,,-0.0,,0.0


# Dual Model

In [5]:
# DUAL MODEL

pi = m.addVariable("pi","positive",domain=[i,j])
phi = m.addVariable("phi","free",domain=i)

dualcons = m.addEquation('dualcons',domain=[i,j])
dualcons[i,j].where[arcs[i,j]]= phi[i].where[~s[i]] + pi[i,j] - phi[j].where[~t[j]] >= Number(1).where[s[i]]

dualflow = m.addModel(
    name="dualflow",
    equations=[dualcons],
    problem=Problem.LP,
    sense=Sense.MIN,
    objective=Sum(arcs, u[arcs]*pi[arcs]),
)

dualflow.solve(solver='cplex',solver_options={'lpmethod': 3, 'netfind': 2, 'preind': 0, 'names': 'no'},output=None)

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,9,11,17,LP,CPLEX,0


# Post Processing

In [6]:
print(f"Objective Function Value: {round(dualflow.objective_value, 4)}\n")
# display(pi.records,phi.records)
print("pi:"); display(pi.records[["i","j","level"]])
print("phi:"); display(phi.records[["i","level"]])

Objective Function Value: 9.0

pi:


,i,j,level
0,1,2,0.0
1,1,3,0.0
2,2,4,0.0
3,3,6,1.0
4,4,7,1.0
5,5,8,0.0
6,6,5,0.0
7,6,8,0.0
8,7,6,0.0
9,7,8,0.0


phi:


,i,level
0,2,-1.0
1,3,-1.0
2,4,-1.0
3,5,0.0
4,6,0.0
5,7,0.0


# Path Flow form of this problem
* Key issue is how to store set of paths
* New data

In [7]:
p = m.addSet('p')
paths = m.addSet('paths',domain=[p,i,j],domain_forwarding=[True,False,False],records=[
    ('p1',1,3), ('p1',3,6), ('p1',6,8),
    ('p2',1,3), ('p2',3,6), ('p2',6,5), ('p2',5,8),
    ('p3',1,2), ('p3',2,4), ('p3',4,7), ('p3',7,8),
    ('p4',1,2), ('p4',2,4), ('p4',4,7), ('p4',7,6), ('p4',6,8),
    ('p5',1,2), ('p5',2,4), ('p5',4,7), ('p5',7,6), ('p5',6,5), ('p5',5,8)] )

In [8]:
# Path Flow Model

In [9]:
f = m.addVariable("f","positive",domain=p,description="path flow")

linkb = m.addEquation('linkb',domain=[i,j])
linkb[arcs[i,j]]= Sum(paths[p,i,j], f[p]) <= u[i,j]

pflow = m.addModel(
    name="pflow",
    equations=[linkb],
    problem=Problem.LP,
    sense=Sense.MAX,
    objective=Sum(p, f[p]),
)

pflow.solve(solver='cplex',solver_options={'lpmethod': 3, 'netfind': 1, 'preind': 0, 'names': 'no'},output=None)

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,9,11,6,LP,CPLEX,0


# Post Processing 

In [10]:
print(f"Objective Function Value: {round(pflow.objective_value, 4)}\n")
print(f"Number of equations: {pflow.num_equations:.0f}\n")
print("f:") 
display(f.records[["p","level"]])    

Objective Function Value: 9.0

Number of equations: 11

f:


,p,level
0,p1,5.0
1,p2,0.0
2,p3,4.0
3,p4,0.0
4,p5,0.0


# Dual of Path Flow Model

In [11]:
pi = m.addVariable("pi","positive",domain=[i,j])
phi = m.addVariable("phi","free",domain=[i])

dualcons = m.addEquation('dualcons',domain=[i,j])
dualcons[arcs[i,j]]= phi[i].where[~s[i]] + pi[i,j] - phi[j].where[~t[j]] >= Number(1).where[s[i]]

dualflow = m.addModel(
    name="dualflow",
    equations=[dualcons],
    problem=Problem.LP,
    sense=Sense.MIN,
    objective=Sum(arcs, u[arcs]*pi[arcs]),
)

dualflow.solve(solver='cplex',solver_options={'lpmethod': 3, 'netfind': 2, 'preind': 0, 'names': 'no'},output=None)

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,9,11,17,LP,CPLEX,0


# Post Processing 

In [12]:
print(f"Objective Function Value: {round(dualflow.objective_value, 4)}\n")
print("pi:"); display(pi.records[["i","j","level"]])
print("phi:"); display(phi.records[["i","level"]])

Objective Function Value: 9.0

pi:


,i,j,level
0,1,2,0.0
1,1,3,0.0
2,2,4,0.0
3,3,6,1.0
4,4,7,1.0
5,5,8,0.0
6,6,5,0.0
7,6,8,0.0
8,7,6,0.0
9,7,8,0.0


phi:


,i,level
0,2,-1.0
1,3,-1.0
2,4,-1.0
3,5,0.0
4,6,0.0
5,7,0.0
